In [1]:
print('ok')

ok


In [1]:
from git import Repo 
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAI
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from dotenv import load_dotenv
import os

In [3]:
%pwd

'c:\\All folders\\Project Files\\Source-Code-Analyzer\\research'

In [4]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"
repo = Repo.clone_from('https://github.com/charan-x16/end-to-end-medical-chatbot-GEN-AI', to_path=repo_path)

In [6]:
blob_loader = FileSystemBlobLoader(repo_path, suffixes=[".py"])

loader = GenericLoader(
    blob_loader=blob_loader, 
    blob_parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)  
)

In [7]:
documents = loader.load()

In [8]:
len(documents)

27

In [9]:
document_loader = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=500,
                                                               chunk_overlap = 20)

In [10]:
text = document_loader.split_documents(documents)

In [11]:
len(text)

98

In [12]:
def download_huggingface_embedding():
     embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
     return embeddings

In [13]:
embedding = download_huggingface_embedding()

C:\Users\chsri\AppData\Local\Temp\ipykernel_20280\3981608823.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\All folders\Project Files\Source-Code-Analyzer\scanalyzer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
vectordb = Chroma.from_documents(text, embedding=embedding, persist_directory='./db')

In [15]:
vectordb.persist()

C:\Users\chsri\AppData\Local\Temp\ipykernel_20280\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [54]:
load_dotenv()

True

In [ ]:
OPENROUTER_API_KEY = os.environ.get('OPENROUTER_API_KEY')
#print(OPENROUTER_API_KEY)

In [77]:

llm = OpenAI(model= 'qwen/qwen3-4b:free',
             base_url="https://openrouter.ai/api/v1",
             api_key=OPENROUTER_API_KEY,
)

In [ ]:

memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr', search_kwargs={'k':8}),
    memory=memory
)


In [82]:
print(rag_chain.input_keys)

['question', 'chat_history']


In [79]:
question = "what is download_hugging_face_embeddings funtion?"


In [ ]:
result = rag_chain(question)
print(result['answer'])